In [13]:
input_file = 'img/asian.svg'
output_file = 'img/asian_parsed.svg'
output_map_name = 'Percent Asian'

In [14]:
line_num = 0
with open(input_file, 'rt') as fin:
    with open(output_file, 'wt') as fout:
        for line in fin:
            # write the stylesheet link after the first xml declaration
            if line.startswith("<?xml"):
                fout.write(line)
                fout.write('<?xml-stylesheet type="text/css" href="../css/styles.css" ?>\n')
            
            # write the script link and define the highlight pattern before the paths
            elif line.startswith("<g color-interpolation"):
                fout.write(f'<title>{output_map_name}</title>')
                fout.write('<script xlink:href="../js/scripts.js" />\n')
                fout.write('<defs>\n')
                fout.write('<g id="background" >\n')
                fout.write('<rect x="0"  y="0" height="5" width="5" fill="#c8c8c8" />\n')
                fout.write('<rect x="5"  y="0" height="5" width="5" fill="#f8f8f8"/>\n')
                fout.write('<rect x="0"  y="5" height="5" width="5" fill="#f8f8f8"/>\n')
                fout.write('<rect x="5"  y="5" height="5" width="5" fill="#c8c8c8" />\n')
                fout.write('</g>\n')
                fout.write('<pattern id="OvalPattern" patternUnits="userSpaceOnUse" width="10" height="10" >\n')
                fout.write('<use xlink:href="#background"/>\n')
                fout.write('</pattern>\n')
                fout.write('</defs>\n')
                fout.write(line)
            
            # use the starting fill color as the class, add an onclick to enable fill change
            else:
                for r in (('fill="#', 'class="rgb'), ('path', f'path id="feature{line_num}" fill="#ffffff" onclick="color(this.id, this.getAttribute(\'class\'))"')):
                    line = line.replace(*r)
                fout.write(line)
            line_num += 1        

In [7]:
p = arcpy.mp.ArcGISProject("Current")
m = p.listMaps()[0]
l = m.listLayers()[0]
sym = l.symbology
[brk.upperBound for brk in sym.renderer.classBreaks]

[0.0024151811385853938, 0.0031615251963977775, 0.003845680531626877, 0.0046775634624481, 0.006688963210702341, 0.00885998821331895, 0.015011309891013777, 0.027218206981393205, 0.11658737024221454]